In [92]:
import cv2  
import numpy as np
import matplotlib.pyplot as plt

In [93]:
video=cv2.VideoCapture("/kaggle/input/video-cv/WhatsApp Video 2024-10-15 at 5.55.03 PM.mp4")
frames_data=[]
while video.isOpened():
    rep,frame=video.read()
    if not rep:
        break
    frames_data.append(frame)
#         break
#         plt.figure()
#         plt.imshow(frame)
#         plt.show()
#     break

In [94]:
frames_data=np.array(frames_data)

In [95]:
frames_data.shape

(124, 850, 478, 3)

# # Quantum CNN

In [96]:
!pip install cirq

In [146]:
from keras.layers import Layer,Dense,Conv2D,MaxPooling2D,Input,Flatten
from keras.models import Model,Sequential
from keras.initializers import HeNormal
from keras.regularizers import l2
from keras.callbacks import EarlyStopping
import cirq
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.utils import to_categorical

In [180]:
data=load_digits()
X, y = data.data, data.target

X = X[y != 2][:, :2]
y = y[y != 2]
X_train,X_test,Y_train,Y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [182]:
encoder=OneHotEncoder(sparse_output=False)
Y_train_encoder=encoder.fit_transform(Y_train.reshape(-1,1))

In [183]:
Y_train.shape

(1296,)

In [184]:
Y_train

array([9, 1, 1, ..., 8, 0, 1])

In [185]:
Y_train2=np.array(Y_train)
Y_train2.shape

(1296,)

In [186]:
np.unique(Y_train)


array([0, 1, 3, 4, 5, 6, 7, 8, 9])

In [187]:
scaler=StandardScaler()
train_data=scaler.fit_transform(X_train)
Y_train_data=to_categorical(Y_train)

In [198]:
train_data

array([[ 0.       , -0.2854575],
       [ 0.       , -0.2854575],
       [ 0.       , -0.2854575],
       ...,
       [ 0.       , -0.2854575],
       [ 0.       , -0.2854575],
       [ 0.       , -0.2854575]])

In [195]:
train_data.shape

(1296, 2)

In [199]:
Y_train_data

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [207]:
def quantum_circuit(data):
    qubit=[cirq.GridQubit(0,1) for i in range(len(data))]
    qubit1 = cirq.GridQubit(0, 0)
    qubit2 = cirq.GridQubit(0, 1)
    circuit=cirq.Circuit()
    for i,value in enumerate(data):
        circuit.append(cirq.rx(value*np.pi).on(qubit[i]))
    circuit.append(cirq.CNOT(qubit1,qubit2))
    circuit.append([cirq.measure(qubit[i]) for i in range(2)])
    return circuit,qubit
def run_quantum_circuit(inputs):
    results=[]
    for input_data in inputs:
        circuit,qubit=quantum_circuit(input_data)
        simulator=cirq.Simulator()
        result=simulator.run(circuit)
        measure0=result.measure(qubit[0]).flatten()[0]
        measure1=result.measure(qubit[1]).flatten()[0]
        combine_data=np.array([measure0,measure1])
        results.append(combine_data)
    return np.array(results,dtype=tf.float32)
class QuantumLayer(Layer):
    def __init__(self):
        super(QuantumLayer, self).__init__()

    def call(self, inputs):
        output = tf.numpy_function(run_quantum_circuit, [inputs], tf.float32)
        output.set_shape((None, 2))  # Adjust shape according to your output
        return output

model2=Sequential([
    tf.keras.Input(shape=(2,)),  # Adjust input shape as needed
    QuantumLayer(),  # Use the quantum layer
    Dense(32, kernel_initializer=HeNormal(), kernel_regularizer=l2(0.01)),
    Dense(1, activation='sigmoid')])

model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model2.summary()



Model: "sequential_35"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ quantum_layer_37 (QuantumLayer) │ (None, 2)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_81 (Dense)                │ (None, 32)             │            96 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_82 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 129 (516.00 B)

 Trainable params: 129 (516.00 B)

 Non-trainable params: 0 (0.00 B)

In [210]:
model2.fit(train_data,Y_train,epochs=5)

Epoch 1/5


UnknownError: Graph execution error:

Detected at node sequential_35_1/quantum_layer_37_1/PyFunc defined at (most recent call last):
  File "/opt/conda/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/opt/conda/lib/python3.10/runpy.py", line 86, in _run_code

  File "/opt/conda/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/opt/conda/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/opt/conda/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/opt/conda/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/opt/conda/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/opt/conda/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/opt/conda/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/opt/conda/lib/python3.10/site-packages/ipykernel/kernelbase.py", line 54